[모델 Softmax 회귀 모델]
1. 절단 정규 분포 (tf.truncated_normal)로 초기화
2. 손실 함수 : Sparse Softmax Cross-Entropy
3. 옵티마이저 : Gradient Descent Optimizer (learning rate=0.5)
4. 학습 epoch=1000
- 학습 데이터 전체를 사용해 한 번에 업데이트하는 GD방식 채택

In [44]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [45]:
# Data
def load_csv(file_path):

    # CSV 파일 로드
    data = np.loadtxt(file_path, delimiter=',', skiprows=1)
    
    # 입력 변수와 목표 변수 분리 ('fetal_health'를 목표 변수로 설정)
    x_data = data[:, :-1]  # 마지막 열 제외 (입력 변수)
    y_data = data[:, -1].astype(int)  # 마지막 열 (목표 변수)
    
    # 목표 변수를 one-hot 인코딩으로 변환
    #with tf.Session() as sess:
    #    y_data = sess.run(tf.one_hot(y_data - 1, depth=3))
    
    y_data = y_data - 1

    return x_data, y_data

def split_data(x_data, y_data):
    # 첫 200개의 데이터를 테스트 데이터로 사용
    x_test = x_data[:200]
    y_test = y_data[:200]
    
    # 나머지 데이터를 학습 및 검증 데이터로 사용
    x_rest = x_data[200:]
    y_rest = y_data[200:]
    
    # 학습 데이터와 검증 데이터로 분할 예시 (전체의 약 12.5%를 검증 데이터로 사용)
    x_train, x_val, y_train, y_val = train_test_split(x_rest, y_rest, test_size=0.125, random_state=42)

    # 표준화 (스케일링)
    sc = StandardScaler()
    x_train = sc.fit_transform(x_train)
    x_val = sc.transform(x_val)
    x_test = sc.transform(x_test)
    
    return x_train, x_val, x_test, y_train, y_val, y_test

In [46]:
x_data, y_data = load_csv('fetal_health.csv')
x_train, x_val, x_test, y_train, y_val, y_test = split_data(x_data, y_data)

In [47]:
x = tf.placeholder(tf.float32, shape=[None, x_train.shape[1]])
y = tf.placeholder(tf.int64, shape=[None])

In [48]:
# Softmax 회귀 모델 정의
W = tf.Variable(tf.truncated_normal([x_train.shape[1], 3]))
b = tf.Variable(tf.zeros([3]))
logits = tf.matmul(x, W) + b
y_pred = tf.nn.softmax(logits)

In [49]:
# 손실 함수 (sparse_softmax_cross_entropy_with_logits 사용)
loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits))

# 옵티마이저 정의 (Gradient Descent Optimizer 사용)
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

In [50]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [51]:
for i in range(1000):
    sess.run(train_step, feed_dict={x: x_train, y: y_train})

In [52]:
correct_prediction = tf.equal(tf.argmax(y_pred,1), y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("정확도(Accuracy): %f" % sess.run(accuracy, feed_dict={x: x_test, y: y_test}))

정확도(Accuracy): 0.845000


In [53]:
sess.close()